Merry Christmas everyone! 

[Advent of Code day 13](http://adventofcode.com/2018/day/13)

OK, this code is going to be hideous 'cos I'm doing it on the train back from [New Model Army](https://newmodelarmy.org) at the Forum, and I'm knackered and not completely sober.

In [1]:
import numpy as np

## Part 1

Let's use a `dict` to represent a cart. Then we can represent each bit of the mine path with a function that returns the next state of the cart.

Here's a representation of the carts in the example:

In [2]:
cart1={'loc':(2, 0),
       'facing': 'E',
       'turn': 'L'
}

cart2={'loc':(9, 3),
       'facing': 'S',
       'turn': 'L'
}

So let's define a function that takes the graphical representation of the tracks, and a cart, and returns a representation of the cart after one step.

Note: this kind of grew. I didn't intend it to be this big and ugly originally.

In [3]:
def move_cart(track, cart_in):
    
    if track[cart_in['loc']]=='|':
        if cart_in['facing']=='N':
            return {'loc':(cart_in['loc'][0], cart_in['loc'][1]-1),
                    'facing':'N',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='S':
            return {'loc':(cart_in['loc'][0], cart_in['loc'][1]+1),
                    'facing':'S',
                    'turn':cart_in['turn']}

    elif track[cart_in['loc']]=='-':

        if cart_in['facing']=='E':
            return {'loc':(cart_in['loc'][0]+1, cart_in['loc'][1]),
                    'facing':'E',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='W':
            return {'loc':(cart_in['loc'][0]-1, cart_in['loc'][1]),
                    'facing':'W',
                    'turn':cart_in['turn']}

    elif track[cart_in['loc']]=='/':

        if cart_in['facing']=='N':
            return {'loc':(cart_in['loc'][0]+1, cart_in['loc'][1]),
                    'facing':'E',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='E':
            return {'loc':(cart_in['loc'][0], cart_in['loc'][1]-1),
                    'facing':'N',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='S':
            return {'loc':(cart_in['loc'][0]-1, cart_in['loc'][1]),
                    'facing':'W',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='W':
            return {'loc':(cart_in['loc'][0], cart_in['loc'][1]+1),
                    'facing':'S',
                    'turn':cart_in['turn']}
        
    elif track[cart_in['loc']]=='\\':

        if cart_in['facing']=='N':
            return {'loc':(cart_in['loc'][0]-1, cart_in['loc'][1]),
                    'facing':'W',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='E':
            return {'loc':(cart_in['loc'][0], cart_in['loc'][1]+1),
                    'facing':'S',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='S':
            return {'loc':(cart_in['loc'][0]+1, cart_in['loc'][1]),
                    'facing':'E',
                    'turn':cart_in['turn']}
        elif cart_in['facing']=='W':
            return {'loc':(cart_in['loc'][0], cart_in['loc'][1]-1),
                    'facing':'N',
                    'turn':cart_in['turn']}

    elif track[cart_in['loc']]=='+':

        if cart_in['facing']=='N':
            if cart_in['turn']=='L':
                return {'loc':(cart_in['loc'][0]-1, cart_in['loc'][1]),
                        'facing':'W',
                        'turn':'F'}
            if cart_in['turn']=='F':
                return {'loc':(cart_in['loc'][0], cart_in['loc'][1]-1),
                        'facing':'N',
                        'turn':'R'}
            if cart_in['turn']=='R':
                return {'loc':(cart_in['loc'][0]+1, cart_in['loc'][1]),
                        'facing':'E',
                        'turn':'L'}

        if cart_in['facing']=='E':
            if cart_in['turn']=='L':
                return {'loc':(cart_in['loc'][0], cart_in['loc'][1]-1),
                        'facing':'N',
                        'turn':'F'}
            if cart_in['turn']=='F':
                return {'loc':(cart_in['loc'][0]+1, cart_in['loc'][1]),
                        'facing':'E',
                        'turn':'R'}
            if cart_in['turn']=='R':
                return {'loc':(cart_in['loc'][0], cart_in['loc'][1]+1),
                        'facing':'S',
                        'turn':'L'}

        if cart_in['facing']=='S':
            if cart_in['turn']=='L':
                return {'loc':(cart_in['loc'][0]+1, cart_in['loc'][1]),
                        'facing':'E',
                        'turn':'F'}
            if cart_in['turn']=='F':
                return {'loc':(cart_in['loc'][0], cart_in['loc'][1]+1),
                        'facing':'S',
                        'turn':'R'}
            if cart_in['turn']=='R':
                return {'loc':(cart_in['loc'][0]-1, cart_in['loc'][1]),
                        'facing':'W',
                        'turn':'L'}

        if cart_in['facing']=='W':
            if cart_in['turn']=='L':
                return {'loc':(cart_in['loc'][0], cart_in['loc'][1]+1),
                        'facing':'S',
                        'turn':'F'}
            if cart_in['turn']=='F':
                return {'loc':(cart_in['loc'][0]-1, cart_in['loc'][1]),
                        'facing':'W',
                        'turn':'R'}
            if cart_in['turn']=='R':
                return {'loc':(cart_in['loc'][0], cart_in['loc'][1]-1),
                        'facing':'N',
                        'turn':'L'}



Try applying to the test input:

In [4]:
test_input_str=r'''/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   '''

It would be useful to have a function that takes the input string and returns a representation of the track and the carts. Note that if there's a cart in the input, then in the track, it needs to be replaced with `|` (for `^` or `v`) or `-` (for `<` or `>`).

In [5]:
def build_initial_state(input_str):
    mine_ar=np.array([[c for c in nl] for nl in input_str.splitlines()]).T
    carts_ls=[]
    for x in range(mine_ar.shape[0]):
        for y in range(mine_ar.shape[1]):
            if mine_ar[x, y]=='>':
                carts_ls.append({'loc':(x, y),
                                 'facing': 'E',
                                 'turn': 'L'})
                mine_ar[x, y]='-'
            if mine_ar[x, y]=='<':
                carts_ls.append({'loc':(x, y),
                                 'facing': 'W',
                                 'turn': 'L'})
                mine_ar[x, y]='-'
            if mine_ar[x, y]=='v':
                carts_ls.append({'loc':(x, y),
                                 'facing': 'S',
                                 'turn': 'L'})
                mine_ar[x, y]='|'
            if mine_ar[x, y]=='^':
                carts_ls.append({'loc':(x, y),
                                 'facing': 'N',
                                 'turn': 'L'})
                mine_ar[x, y]='|'
    return {'map':mine_ar, 'carts':carts_ls}

I quite like the way they display the mine and carts, so let's write a `display` function:

In [6]:
def display(mine_ar, carts_ls):
    "Hacky display that doesn't consider crashes"
    for y in range(mine_ar.shape[1]):
        for x in range(mine_ar.shape[0]):
            cart_here=False
            for c in carts_ls:
                if c['loc']==(x, y):
                    cart_here=True
                    if c['facing']=='N':
                        print('^', end='')
                        break
                    elif c['facing']=='E':
                        print('>', end='')
                        break
                    elif c['facing']=='S':
                        print('v', end='')
                        break
                    elif c['facing']=='W':
                        print('<', end='')
                        break
            if not cart_here:
                print(mine_ar[x, y], end='')
        print()

In [7]:
s_dict=build_initial_state(test_input_str)

display(s_dict['map'], s_dict['carts'])

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   


OK, so far so good. Now, let's try some next steps. We move the carts in order of y coordinate then x coordinate:

In [8]:
sorted(s_dict['carts'], key=lambda x:(x['loc'][1], x['loc'][0]))

[{'facing': 'E', 'loc': (2, 0), 'turn': 'L'},
 {'facing': 'S', 'loc': (9, 3), 'turn': 'L'}]

If there's a collision, I'll return the coordinates of the collision, otherwise the list of carts. Need to check for the collision both before and after adding to `carts_out` so that we don't get eg. `--><--` $\rightarrow$ `--<>--`.

In [9]:
def move_carts(map_in, carts_ls):
    carts_out=[]
    for cart in sorted(carts_ls, key=lambda x:(x['loc'][1], x['loc'][0])):
        if any([c['loc']==cart['loc'] for c in carts_out]):
            return c['loc']
        cart1=move_cart(map_in, cart)
        if any([c['loc']==cart1['loc'] for c in carts_out]):
            return cart1['loc']
        carts_out.append(cart1)
    return carts_out

Run through a few iterations and make sure it's working OK:

In [10]:
s_dict=build_initial_state(test_input_str)

mc=lambda x:move_carts(s_dict['map'], x)
carts_ls=s_dict['carts']

for i in range(15):
    display(s_dict['map'], carts_ls)
    for c in carts_ls:
        print(c)
    print()
    
    carts_ls=mc(carts_ls)
    if type(carts_ls)==tuple:
        print('Collide at {}'.format(carts_ls))
        break

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
{'loc': (2, 0), 'facing': 'E', 'turn': 'L'}
{'loc': (9, 3), 'facing': 'S', 'turn': 'L'}

/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-v--/
  \------/   
{'loc': (3, 0), 'facing': 'E', 'turn': 'L'}
{'loc': (9, 4), 'facing': 'S', 'turn': 'L'}

/--->        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   
{'loc': (4, 0), 'facing': 'E', 'turn': 'L'}
{'loc': (10, 4), 'facing': 'E', 'turn': 'F'}

/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   
{'loc': (4, 1), 'facing': 'S', 'turn': 'L'}
{'loc': (11, 4), 'facing': 'E', 'turn': 'F'}

/---\        
|   |  /----\
| /-v--+-\  |
| | |  | |  |
\-+-/  \-+-->
  \------/   
{'loc': (4, 2), 'facing': 'S', 'turn': 'L'}
{'loc': (12, 4), 'facing': 'E', 'turn': 'F'}

/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   
{'loc': (5, 2), 'facing': 'E', 'turn': 'F'}
{'lo

OK, and now do it with our input:

In [11]:
puzzle_input_str=open('inputs/day13').read()

In [12]:
s_dict=build_initial_state(puzzle_input_str)

carts_ls=s_dict['carts']

while True:
    carts_ls=move_carts(s_dict['map'], carts_ls)
    if type(carts_ls)==tuple:
        print('Collide at {}'.format(carts_ls))
        break

Collide at (38, 57)


Correct.

## Part 2

Right, let's assume that we can just leave it running and see what happens.

To run this problem, we (just?) need to remove the carts when they collide. So we need an updated version of `move_carts`:

In [13]:
def move_carts_1(map_in, carts_ls):
    carts_out=[]
    for cart in sorted(carts_ls, key=lambda x:(x['loc'][1], x['loc'][0])):
        crash=False
        for (i, c) in enumerate(carts_out):
            if c['loc']==cart['loc']:
                carts_out.pop(i)
                crash=True
        cart1=move_cart(map_in, cart)
        if not crash:
            for (i, c) in enumerate(carts_out):
                if c['loc']==cart1['loc']:
                    carts_out.pop(i)
                    crash=True
        if not crash:
            carts_out.append(cart1)
    return carts_out

In [14]:
test_input_2_str=r'''/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/'''

In [15]:
s_dict=build_initial_state(test_input_2_str)

display(s_dict['map'], s_dict['carts'])

/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/


In [16]:
display(s_dict['map'], move_carts_1(s_dict['map'], s_dict['carts']))

/---\  
|   |  
| <-+-\
| | | |
\-+-/ |
  |   |
  <--->


In [17]:
display(s_dict['map'], move_carts_1(s_dict['map'], move_carts_1(s_dict['map'], s_dict['carts'])))

/---\  
|   |  
| /-+-\
| v | |
\-+-/ |
  ^   ^
  \---/


In [18]:
display(s_dict['map'], move_carts_1(s_dict['map'], move_carts_1(s_dict['map'], move_carts_1(s_dict['map'], s_dict['carts']))))

/---\  
|   |  
| /-+-\
| | | |
\-+-/ ^
  |   |
  \---/


That seems to be working OK. So we want a loop that terminates when there's only one cart left.

In [19]:
s_dict=build_initial_state(test_input_2_str)

carts_ls=s_dict['carts']

while len(carts_ls)>1:
    carts_ls=move_carts_1(s_dict['map'], carts_ls)
    
carts_ls[0]['loc']

(6, 4)

Works on the test case. How about the puzzle input? I'll just get it to print when the number of carts is reduced.

In [20]:
s_dict=build_initial_state(puzzle_input_str)

i=0

carts_ls=s_dict['carts']

l=len(carts_ls)

while len(carts_ls)>1:
    carts_ls=move_carts_1(s_dict['map'], carts_ls)
    if len(carts_ls)<l:
        print('{}\t{}'.format(i, l))
        l=len(carts_ls)
    i+=1
    
carts_ls[0]['loc']

210	17
286	15
546	13
557	11
889	9
2110	7
2970	5
15207	3


(4, 92)

Correct. And runs quicker than I was expecting.